# Limpieza de datos del dataset de coches de segunda mano de Kaggle.

## 1. Información de los datos Scrapeados.

En este Notebook se limpian los datos del dataset de coches de segunda mano de kaggle que se usará en caso de no contar con suficientes vehículos scrapeados, para crear un mejor modelo.

Por ello, los datos presentes en este dataset se transformarán levemente para que tengan las mismas features y formato que los datos extraídos mediante el WebScraping que se ha realizado de [coches.net](coches.net).

Lo primero que haremos es cargar algunos vehículos de la tabla TX_VEHICULOS_SEG_MANO de la base de datos en la que guardamos los vehículos scrapeados para ver los campos de la tabla y su formato:

In [13]:
import pandas as pd
import sqlite3

# Conexión a la BBDD:
con = sqlite3.connect("../include/db_vehiculos.db")

# Vamos a ver qué nos devuelve un select sobre la tabla:
query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    origen_anuncio = 'WebScraping'
    --and marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')
order by
    fecha_carga desc
;
"""

# Leemos los resultados en un DataFrame de pandas
df_db = pd.read_sql_query(query, con)

# Mostramos los primeros registros:
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60545114,CITROEN,c4 picasso,7999.0,Diésel,2014,195000,115.0,5.0,5.0,...,Monovolumen,1560.0,negro,Barcelona,B (amarilla),WebScraping,2025-05-27,202505,2025-05-27 10:50:08,https://www.coches.net/citroen-c4-picasso-16-e...
1,60657064,OPEL,corsa,11985.0,Gasolina,2020,32325,100.0,5.0,5.0,...,Berlina,1364.0,blanco,Sta. C. Tenerife,C (verde),WebScraping,2025-05-27,202505,2025-05-27 10:49:44,https://www.coches.net/opel-corsa-14-turbo-74k...
2,60656553,OPEL,corsa,14990.0,Gasolina,2022,63540,130.0,5.0,5.0,...,Berlina,1199.0,azul,Valencia,None,WebScraping,2025-05-27,202505,2025-05-27 10:48:33,https://www.coches.net/opel-corsa-12t-xht-96kw...
3,60659620,CITROEN,berlingo,9990.0,Gasolina,2018,110000,110.0,5.0,5.0,...,Monovolumen,1199.0,gris,Sta. C. Tenerife,None,WebScraping,2025-05-27,202505,2025-05-27 10:47:11,https://www.coches.net/citroen-berlingo-talla-...
4,60659465,AUDI,a3,29150.0,Híbrido,2020,82351,150.0,5.0,5.0,...,Berlina,1498.0,azul,Madrid,ECO (azul/verde),WebScraping,2025-05-27,202505,2025-05-27 10:37:06,https://www.coches.net/audi-a3-sportback-genui...


In [14]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002 entries, 0 to 5001
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        5002 non-null   int64  
 1   marca                5002 non-null   object 
 2   modelo               5002 non-null   object 
 3   precio               5002 non-null   float64
 4   combustible          4820 non-null   object 
 5   anyo_vehiculo        5002 non-null   int64  
 6   kilometraje          5002 non-null   int64  
 7   potencia             4885 non-null   float64
 8   num_puertas          4890 non-null   float64
 9   num_plazas           4886 non-null   float64
 10  tipo_cambio          5002 non-null   object 
 11  tipo_vehiculo        5002 non-null   object 
 12  cilindrada_motor     4864 non-null   float64
 13  color                4830 non-null   object 
 14  provincia            5002 non-null   object 
 15  etiqueta_eco         3306 non-null   o

Vemos que la tabla TX_VEHICULOS_SEG_MANO tiene 21 columnas, en la que cada registro se identifica únicamente por el pk_anuncio_id.

Además, tiene varias columnas tanto numéricas como categóricas que servirán para features del modelo predictivo, la columna precio será el target a predecir. 

También existe una columna llamada ```"origen_anuncio"``` que nos servirá para distinguir entre anuncios scrapeados o provenientes del dataset de Kaggle en caso de combinar ambos orígenes de datos y, por último, hay columnas informativas como las columnas: ```["fecha_publicacion", "anyomes_publicacion", "fecha_carga", "url"]```.

## 2. Información de los datos del Dataset de Kaggle.

Ahora vamos a ver el tipo de datos que contiene el dataset de Kaggle.

Importamos los datos desde el siguiente link:

[https://www.kaggle.com/datasets/datamarket/venta-de-coches](https://www.kaggle.com/datasets/datamarket/venta-de-coches)

Creo que el link no funciona actualmente, por lo que importaremos los datos desde local, ya que los tengo descargados.

In [15]:
# Importamos las librerías necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
df = pd.read_csv("../include/data/raw/coches-de-segunda-mano-sample.csv")
df.head()

,url,company,make,model,version,price,price_financed,fuel,year,kms,...,doors,shift,color,photos,is_professional,dealer,province,country,publish_date,insert_date
0,e158ae0ca53119ca199c28c36b5c2fcd,9881bcdd5a0ad4733037b3fb25e69c3a,SEAT,Toledo,SEAT Toledo 4p.,950,NaN,Diésel,2000.0,227000,...,4,Manual,Verde,5,False,0f4bb8455d27349b8273109b66a847f3,Navarra,Spain,2020-12-18 10:47:13,2021-01-15 00:00:00
1,ff267ebb7e700246f47f84f3db660b4b,9881bcdd5a0ad4733037b3fb25e69c3a,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,NaN,Gasolina,2017.0,50071,...,5,Manual,Blanco,6,True,Autos Raymara,Tenerife,Spain,2021-01-02 11:25:40,2021-01-15 00:00:00
2,de4b02db28ea7786c622b969be10c7c7,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,...,4,Manual,Blanco,10,True,Auto 96,Barcelona,Spain,2020-12-16 10:51:45,2021-01-15 00:00:00
3,0449972a4d07594acf92e9a7dd28b39c,9881bcdd5a0ad4733037b3fb25e69c3a,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,NaN,Diésel,2014.0,120000,...,4,Manual,Blanco,9,True,Inniauto,Navarra,Spain,2020-11-25 11:09:14,2021-01-15 00:00:00
4,12c4fa49bd4fdf23f19ecf396d3f02ef,9881bcdd5a0ad4733037b3fb25e69c3a,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,...,2,Manual,Blanco,4,True,"Autofleet España,s.l",Sevilla,Spain,2021-01-12 20:00:34,2021-01-15 00:00:00


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              50000 non-null  object 
 1   company          50000 non-null  object 
 2   make             49998 non-null  object 
 3   model            49995 non-null  object 
 4   version          50000 non-null  object 
 5   price            50000 non-null  int64  
 6   price_financed   23563 non-null  float64
 7   fuel             49954 non-null  object 
 8   year             49998 non-null  float64
 9   kms              50000 non-null  int64  
 10  power            41472 non-null  float64
 11  doors            50000 non-null  int64  
 12  shift            49889 non-null  object 
 13  color            49406 non-null  object 
 14  photos           50000 non-null  int64  
 15  is_professional  50000 non-null  bool   
 16  dealer           50000 non-null  object 
 17  province    

Vemos que el Dataset de Kaggle cuenta con 50.000 registros. Vamos a ver cuántos registros tiene por marca:

In [18]:
# Número de coches por marca en el df:

num_coches_marca_df = df.groupby("make")["make"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_marca_df)

make
VOLKSWAGEN        4624
BMW               4510
MERCEDES-BENZ     4227
AUDI              3745
PEUGEOT           3314
RENAULT           3127
FORD              3116
OPEL              2924
CITROEN           2855
SEAT              2604
NISSAN            1634
TOYOTA            1562
FIAT              1398
HYUNDAI           1149
KIA               1118
VOLVO              823
MINI               811
LAND-ROVER         595
SKODA              574
PORSCHE            517
MAZDA              490
HONDA              464
JAGUAR             339
MITSUBISHI         334
ALFA ROMEO         302
JEEP               299
LEXUS              282
DACIA              277
SUZUKI             252
CHEVROLET          224
SSANGYONG          216
SMART              196
CHRYSLER           126
DS                 125
SUBARU              99
ABARTH              96
INFINITI            78
SAAB                73
LANCIA              62
MASERATI            62
IVECO               48
DAEWOO              45
ROVER               36
FERRAR

Observamos que para la marca que más vehículos hay registrados es ```VolksWagen```, con 4.624 registros.

Como solo estamos scrapeando vehículos de las siguientes 4 marcas (ya que tampoco se pueden scrapear más rápido sin ser detectados y para que podamos realizar un modelo que funcione bien para estas 4), nos vamos a quedar sólo con los vehículos de estas marcas:

```['HYUNDAI', 'KIA', 'NISSAN', 'TOYOTA']```

In [19]:
df = df.loc[df["make"].isin(['HYUNDAI', 'KIA', 'NISSAN', 'TOYOTA'])]

df.head()

,url,company,make,model,version,price,price_financed,fuel,year,kms,...,doors,shift,color,photos,is_professional,dealer,province,country,publish_date,insert_date
9,13b881bdd31a0387edd64edd726e8c93,9881bcdd5a0ad4733037b3fb25e69c3a,NISSAN,NOTE,NISSAN NOTE 5p. 1.2G 80CV Acenta NissanConnect...,6300,NaN,Gasolina,2016.0,77000,...,5,Manual,Blanco,6,False,8a3b0eb4d055a275274895319688e80a,Barcelona,Spain,2020-12-14 17:39:46,2021-01-15 00:00:00
35,9df64689ac5768546723b844d76a1b9b,9881bcdd5a0ad4733037b3fb25e69c3a,KIA,Sportage,KIA Sportage 5p.,4500,NaN,Diésel,2008.0,128000,...,5,Manual,Gris / Plata,5,False,464e07afc9e46359fb480839150595c5,Huelva,Spain,2021-01-14 05:38:48,2021-01-15 00:00:00
39,0d17809a4a1de7ef2c63f342f306713c,9881bcdd5a0ad4733037b3fb25e69c3a,KIA,XCeed,KIA XCeed 1.6 GDi PHEV 104kW 141CV eMotion 5p.,32300,28800.0,Híbrido enchufable,2021.0,2800,...,5,Manual,Cassa White (WD),6,True,Talleres M Gallego,Murcia,Spain,2020-12-17 08:46:44,2021-01-15 00:00:00
50,a19d08cbe35ce508121bd5613d46455b,9881bcdd5a0ad4733037b3fb25e69c3a,NISSAN,NOTE,NISSAN NOTE 5p. 1.2G 80CV NTEC Pack Comfort 5p.,7900,NaN,Gasolina,2015.0,54200,...,5,Manual,Blanco,2,False,c32d65ca9197efa4d8c2df006034a6ef,Pontevedra,Spain,2020-12-18 09:22:37,2021-01-15 00:00:00
72,02916bf1509d2988bc4f5cb9e2551799,9881bcdd5a0ad4733037b3fb25e69c3a,HYUNDAI,i40,HYUNDAI i40 1.6 GDI GLS 135cv 4p.,8400,NaN,Gasolina,2012.0,107000,...,4,Manual,Gris / Plata,14,True,Central Seminuevos Km0,Álava,Spain,2021-01-08 22:39:01,2021-01-15 00:00:00


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5463 entries, 9 to 49995
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              5463 non-null   object 
 1   company          5463 non-null   object 
 2   make             5463 non-null   object 
 3   model            5463 non-null   object 
 4   version          5463 non-null   object 
 5   price            5463 non-null   int64  
 6   price_financed   3358 non-null   float64
 7   fuel             5446 non-null   object 
 8   year             5462 non-null   float64
 9   kms              5463 non-null   int64  
 10  power            4878 non-null   float64
 11  doors            5463 non-null   int64  
 12  shift            5452 non-null   object 
 13  color            5425 non-null   object 
 14  photos           5463 non-null   int64  
 15  is_professional  5463 non-null   bool   
 16  dealer           5463 non-null   object 
 17  province         5

In [21]:
# Comprobamos el número de marcas ahora:

num_coches_marca_df = df.groupby("make")["make"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_marca_df)

make
NISSAN     1634
TOYOTA     1562
HYUNDAI    1149
KIA        1118
Name: make, dtype: int64


Nos hemos quedado con **5.463 registros, pertenecientes a 4 marcas**.

Si comparamos los campos presentes en ambos DataFrames, vemos que coinciden varios de ellos. Vamos a **eliminar** los siguientes campos de este DataFrame importado de Kaggle, ya que no existen en los registros scrapeados:

- ```company```
- ```version```
- ```price_financed```
- ```photos```
- ```is_professional```
- ```dealer```
- ```country```

In [22]:
df.drop(columns=["company", "version", "price_financed", "photos", "is_professional", "dealer", "country"],
         inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5463 entries, 9 to 49995
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           5463 non-null   object 
 1   make          5463 non-null   object 
 2   model         5463 non-null   object 
 3   price         5463 non-null   int64  
 4   fuel          5446 non-null   object 
 5   year          5462 non-null   float64
 6   kms           5463 non-null   int64  
 7   power         4878 non-null   float64
 8   doors         5463 non-null   int64  
 9   shift         5452 non-null   object 
 10  color         5425 non-null   object 
 11  province      5462 non-null   object 
 12  publish_date  5463 non-null   object 
 13  insert_date   5463 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 640.2+ KB


Vemos que ahora nos quedamos con 15 columnas.

Además, también vamos a renombrar los siguientes campos para que coincidan con los de nuestra tabla original:

- ```make``` --> ```marca```
- ```model``` --> ```modelo```
- ```price``` --> ```precio```
- ```fuel``` --> ```combustible```
- ```year``` --> ```anyo_vehiculo```
- ```kms``` --> ```kilometraje```
- ```power``` --> ```potencia```
- ```doors``` --> ```num_puertas```
- ```shift``` --> ```tipo_cambio```
- ```province``` --> ```provincia```
- ```publish_date``` --> ```fecha_publicacion```
- ```insert_date``` --> ```fecha_carga```

También creamos los siguientes campos para poder insertar estos registros en la base de datos:

- ```pk_anuncio_id```: Secuencial que identifica el vehículo. No coincide con el de los vehículos scrapeados, ya tenemos 35.046 en este dataset y los scrapeados empiezan en 50000000.
- ```origen_anuncio```: Tendrá el valor "dataset kaggle" para todos estos registros importados desde el Dataset.
- ```anyomes_publicacion```: Año y mes de la fecha de publicación en formato 'YYYYMM'.

Creamos también los siguientes campos, aunque los dejaremos como *null* para los datos de este origen:

- ```num_plazas```: *null*
- ```tipo_vehiculo```: *null*
- ```cilindrada_motor```: *null*
- ```etiqueta_eco```: *null*

In [24]:
import pandas as pd

# Renombramos las columnas:
df = df.rename(columns={
    'make': 'marca',
    'model': 'modelo',
    'price': 'precio',
    'fuel': 'combustible',
    'year': 'anyo_vehiculo',
    'kms': 'kilometraje',
    'power': 'potencia',
    'doors': 'num_puertas',
    'shift': 'tipo_cambio',
    'province': 'provincia',
    'publish_date': 'fecha_publicacion',
    'insert_date': 'fecha_carga'
})

# Campo 'pk_anuncio_id':
df['pk_anuncio_id'] = range(100000, 100000 + len(df))

# Campo 'origen_anuncio':
df['origen_anuncio'] = 'dataset kaggle'

# Campo 'anyomes_publicacion' en formato YYYYMM:
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], errors='coerce')
df['anyomes_publicacion'] = df['fecha_publicacion'].dt.strftime('%Y%m')

# Campos nulos:
df['num_plazas'] = None
df['tipo_vehiculo'] = None
df['cilindrada_motor'] = None
df['etiqueta_eco'] = None

# Conversión de tipos:
df['fecha_publicacion'] = df['fecha_publicacion'].dt.strftime('%Y-%m-%d')
df['anyomes_publicacion'] = df['anyomes_publicacion'].astype('Int64')  # Permite nulos
df['precio'] = df['precio'].astype(float)

# Reordenamos las columnas:
orden_columnas = [
    'pk_anuncio_id',
    'marca',
    'modelo',
    'precio',
    'combustible',
    'anyo_vehiculo',
    'kilometraje',
    'potencia',
    'num_puertas',
    'num_plazas',
    'tipo_cambio',
    'tipo_vehiculo',
    'cilindrada_motor',
    'color',
    'provincia',
    'etiqueta_eco',
    'origen_anuncio',
    'fecha_publicacion',
    'anyomes_publicacion',
    'fecha_carga',
    'url'
]

df = df[orden_columnas]


df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,...,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
9,100000,NISSAN,NOTE,6300.0,Gasolina,2016.0,77000,80.0,5,None,...,None,None,Blanco,Barcelona,None,dataset kaggle,2020-12-14,202012,2021-01-15 00:00:00,13b881bdd31a0387edd64edd726e8c93
35,100001,KIA,Sportage,4500.0,Diésel,2008.0,128000,NaN,5,None,...,None,None,Gris / Plata,Huelva,None,dataset kaggle,2021-01-14,202101,2021-01-15 00:00:00,9df64689ac5768546723b844d76a1b9b
39,100002,KIA,XCeed,32300.0,Híbrido enchufable,2021.0,2800,141.0,5,None,...,None,None,Cassa White (WD),Murcia,None,dataset kaggle,2020-12-17,202012,2021-01-15 00:00:00,0d17809a4a1de7ef2c63f342f306713c
50,100003,NISSAN,NOTE,7900.0,Gasolina,2015.0,54200,80.0,5,None,...,None,None,Blanco,Pontevedra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,a19d08cbe35ce508121bd5613d46455b
72,100004,HYUNDAI,i40,8400.0,Gasolina,2012.0,107000,135.0,4,None,...,None,None,Gris / Plata,Álava,None,dataset kaggle,2021-01-08,202101,2021-01-15 00:00:00,02916bf1509d2988bc4f5cb9e2551799


In [54]:
# Comprobamos que el id se genera bien:

#df["pk_anuncio_id"].value_counts()

Comprobamos como ha quedado el df asignado al Dataset de Kaggle:

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5463 entries, 9 to 49995
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        5463 non-null   int64  
 1   marca                5463 non-null   object 
 2   modelo               5463 non-null   object 
 3   precio               5463 non-null   float64
 4   combustible          5446 non-null   object 
 5   anyo_vehiculo        5462 non-null   float64
 6   kilometraje          5463 non-null   int64  
 7   potencia             4878 non-null   float64
 8   num_puertas          5463 non-null   int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          5452 non-null   object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                5425 non-null   object 
 14  provincia            5462 non-null   object 
 15  etiqueta_eco         0 non-null      objec

Ahora que ya tenemos las columnas que equivalen a las que obtenemos mediante WebScraping, vamos a compararlas, ya que todavía hay modificaciones que realizar para poder combinar ambas fuentes de datos.

Recuperamos las columnas del ```df_db```, que contiene los vehículos del WebScraping:

In [56]:
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        731 non-null    int64  
 1   marca                731 non-null    object 
 2   modelo               731 non-null    object 
 3   precio               731 non-null    float64
 4   combustible          718 non-null    object 
 5   anyo_vehiculo        731 non-null    int64  
 6   kilometraje          731 non-null    int64  
 7   potencia             728 non-null    float64
 8   num_puertas          731 non-null    int64  
 9   num_plazas           728 non-null    float64
 10  tipo_cambio          731 non-null    object 
 11  tipo_vehiculo        731 non-null    object 
 12  cilindrada_motor     719 non-null    float64
 13  color                722 non-null    object 
 14  provincia            731 non-null    object 
 15  etiqueta_eco         564 non-null    obj

In [26]:
# Comprobamos el número de modelos ahora:

num_coches_modelo_df = df.groupby("modelo")["modelo"].count().sort_values(ascending=False)

pd.set_option("display.max_rows", None)
print(num_coches_modelo_df)

modelo
QASHQAI                    594
Auris                      335
Sportage                   229
JUKE                       222
TUCSON                     210
Yaris                      205
i30                        178
X-TRAIL                    172
Rav4                       166
C-HR                       157
Micra                      146
ceed                       141
Corolla                    124
Avensis                    121
Carens                     115
i20                        108
Kona                       102
Aygo                        98
Stonic                      88
i40                         84
Verso                       84
Picanto                     71
LEAF                        70
i10                         66
ix35                        65
Rio                         63
IONIQ                       63
XCeed                       61
Niro                        60
Santa Fe                    56
Almera                      54
Sorento                     53
C

## Transformaciones para homogeneizar los datos de ambos orígenes.

Ahora vamos a ver las diferencias entre columnas de ambos DataFrames, para corregirlas y que queden lo más homogéneas posibles en la base de datos.

Coches dataset kaggle:

In [27]:
pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
9,100000,NISSAN,NOTE,6300.0,Gasolina,2016.0,77000,80.0,5,None,Manual,None,None,Blanco,Barcelona,None,dataset kaggle,2020-12-14,202012,2021-01-15 00:00:00,13b881bdd31a0387edd64edd726e8c93
35,100001,KIA,Sportage,4500.0,Diésel,2008.0,128000,NaN,5,None,Manual,None,None,Gris / Plata,Huelva,None,dataset kaggle,2021-01-14,202101,2021-01-15 00:00:00,9df64689ac5768546723b844d76a1b9b
39,100002,KIA,XCeed,32300.0,Híbrido enchufable,2021.0,2800,141.0,5,None,Manual,None,None,Cassa White (WD),Murcia,None,dataset kaggle,2020-12-17,202012,2021-01-15 00:00:00,0d17809a4a1de7ef2c63f342f306713c
50,100003,NISSAN,NOTE,7900.0,Gasolina,2015.0,54200,80.0,5,None,Manual,None,None,Blanco,Pontevedra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,a19d08cbe35ce508121bd5613d46455b
72,100004,HYUNDAI,i40,8400.0,Gasolina,2012.0,107000,135.0,4,None,Manual,None,None,Gris / Plata,Álava,None,dataset kaggle,2021-01-08,202101,2021-01-15 00:00:00,02916bf1509d2988bc4f5cb9e2551799


Coches WebScraping coches.net:

In [63]:
pd.set_option('display.max_columns', None)
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,58168,75.0,5,5.0,Manual,Berlina,1199.0,Rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022,18652,100.0,5,5.0,Manual,Berlina,1199.0,Azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022,27493,100.0,5,5.0,Manual,Berlina,1199.0,Gris / Plata,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,49466,75.0,5,5.0,Manual,Berlina,1199.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018,147508,95.0,5,5.0,Manual,Berlina,1598.0,Blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


Comparamos primero los valores de la columna ```"combustible"```:

In [28]:
df["combustible"].value_counts()

combustible
Diésel                2380
Gasolina              2163
Híbrido                704
Eléctrico              133
Híbrido enchufable      66
Name: count, dtype: int64

In [61]:
df_db["combustible"].value_counts()

combustible
Diesel                343
Gasolina              324
Híbrido                36
Eléctrico               9
Híbrido Enchufable      6
Name: count, dtype: int64

Vemos que son ligeramente distintas. Lo que haremos será corregir la carga del WebScraping para ajustarlos a los valores del dataset de kaggle.

Vamos con los valores de la columna ```"color"```:

In [29]:
df["color"].value_counts()

color
Blanco                                                                   1527
Gris / Plata                                                             1105
Negro                                                                     601
Azul                                                                      438
Rojo                                                                      348
Marrón                                                                     84
Verde                                                                      79
Granate                                                                    67
Beige                                                                      66
Amarillo                                                                   44
Blanco (BLANCO)                                                            38
Naranja                                                                    33
Blanco (BLANCO PERLADO)                                   

In [65]:
df_db["color"].value_counts()

color
Blanco          281
Gris / Plata    133
Negro           114
Azul             90
Rojo             79
Naranja           9
Marrón            5
Beige             5
Verde             3
Amarillo          3
Name: count, dtype: int64

Aquí vemos que los colores del dataset de kaggle son muy dispares. De momento, lo que haremos es quedarnos con la primera palabra en cada string y ponerlo en minúscula en ambos DataFrames:

In [30]:
# Dataset kaggle:

import numpy as np

df["color"] = df["color"].str.split().str[0].str.lower()

# Reemplazamos con NaN si no empieza por una letra o número:
df["color"] = df["color"].where(df["color"].str.match(r'^[a-z0-9]', na=False), np.nan)


df["color"].value_counts()

color
blanco       1822
gris         1318
negro         678
azul          550
rojo          449
marrón         90
verde          84
beige          79
granate        76
amarillo       53
naranja        41
violeta        13
326             9
wd              8
ud              7
040             5
hw2             4
nka             4
polar           4
1d              4
pyw             4
y3y             3
cassa           3
u3s             3
qab             3
tcw             3
1f7             3
platinum        3
dark            2
11              2
cb7             2
runway          2
swp             2
signal          2
frd             2
pulse           2
im              2
mzh             2
d5u             2
209             2
white           2
1g3             2
rah             2
070             2
xn3             2
kad             2
clear           2
x-blanco        2
qm1             2
s6g             2
z11             2
k3y             2
chalk           2
aurora          2
aa9             2
micr

In [ ]:
# Dataset WebScraping:

import numpy as np

df_db["color"] = df_db["color"].str.split().str[0].str.lower()

# Reemplazamos con NaN si no empieza por una letra o número:
df_db["color"] = df_db["color"].where(df_db["color"].str.match(r'^[a-z0-9]', na=False), np.nan)


df_db["color"].value_counts()

color
blanco      281
gris        133
negro       114
azul         90
rojo         79
naranja       9
marrón        5
beige         5
verde         3
amarillo      3
Name: count, dtype: int64

De momento vamos a dejarlo así, aunque quizás hagamos alguna transformación más para el modelo para los colores de los datos del Dataset de Kaggle.

Vamos a printear de nuevo los DataFrames a ver si hay alguna columna más que modificar:

In [31]:
#Dataset Kaggle:

pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
9,100000,NISSAN,NOTE,6300.0,Gasolina,2016.0,77000,80.0,5,None,Manual,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-14,202012,2021-01-15 00:00:00,13b881bdd31a0387edd64edd726e8c93
35,100001,KIA,Sportage,4500.0,Diésel,2008.0,128000,NaN,5,None,Manual,None,None,gris,Huelva,None,dataset kaggle,2021-01-14,202101,2021-01-15 00:00:00,9df64689ac5768546723b844d76a1b9b
39,100002,KIA,XCeed,32300.0,Híbrido enchufable,2021.0,2800,141.0,5,None,Manual,None,None,cassa,Murcia,None,dataset kaggle,2020-12-17,202012,2021-01-15 00:00:00,0d17809a4a1de7ef2c63f342f306713c
50,100003,NISSAN,NOTE,7900.0,Gasolina,2015.0,54200,80.0,5,None,Manual,None,None,blanco,Pontevedra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,a19d08cbe35ce508121bd5613d46455b
72,100004,HYUNDAI,i40,8400.0,Gasolina,2012.0,107000,135.0,4,None,Manual,None,None,gris,Álava,None,dataset kaggle,2021-01-08,202101,2021-01-15 00:00:00,02916bf1509d2988bc4f5cb9e2551799


In [73]:
# Dataset WebScraping:

pd.set_option('display.max_columns', None)
df_db.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,60579674,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,58168,75.0,5,5.0,Manual,Berlina,1199.0,rojo,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:58:23,https://www.coches.net/opel-corsa-12-xel-55kw-...
1,60579673,OPEL,Corsa 1.2T XHL GSLine,14510.0,Gasolina,2022,18652,100.0,5,5.0,Manual,Berlina,1199.0,azul,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:56,https://www.coches.net/opel-corsa-12t-xhl-74kw...
2,60579672,OPEL,Corsa 1.2T XHL GSLine,13950.0,Gasolina,2022,27493,100.0,5,5.0,Manual,Berlina,1199.0,gris,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:57:34,https://www.coches.net/opel-corsa-12t-xhl-74kw...
3,60579678,OPEL,Corsa 1.2 XEL Edition,12290.0,Gasolina,2022,49466,75.0,5,5.0,Manual,Berlina,1199.0,blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:49,https://www.coches.net/opel-corsa-12-xel-55kw-...
4,60580266,SEAT,Ibiza 1.6 TDI Style,11499.0,Diesel,2018,147508,95.0,5,5.0,Manual,Berlina,1598.0,blanco,Valencia,C (verde),WebScraping,2025-05-16,202505.0,2025-05-16 15:56:26,https://www.coches.net/seat-ibiza-16-tdi-70kw-...


Los modelos también son distintos, pero lo que haremos es hacer una lista con todos los modelos del Dataset de Kaggle, que es más genérico, e identificar los modelos del WebScraping con estos. Si no es posible en alguno, dejaremos el modelo específico del WebScrapping.

Las siguientes transformaciones del dataset de coches de WebScraping serán incorporadas en el script de carga del scrapeo, para que los datos se carguen ya correctamente:

- modelo in modelos del dataset --> coge ese, si no, guarda el string.
- combustible en este formato: [Diésel, Gasolina, Híbrido enchufable, Eléctrico, Híbrido]
- color --> quedarse solo con el primero y en minúscula.




## Subida del DataFrame de Kaggle a la BBDD:

Ahora que tenemos el DataFrame de Kaggle en un formato compatible con el de los vehículos scrapeados, vamos a guardar los registros para que todos estén en la misma tabla.

Primero comprobamos el estado actual del DataFrame de Kaggle:

In [32]:
#Dataset Kaggle:

pd.set_option('display.max_columns', None)
df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
9,100000,NISSAN,NOTE,6300.0,Gasolina,2016.0,77000,80.0,5,None,Manual,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-14,202012,2021-01-15 00:00:00,13b881bdd31a0387edd64edd726e8c93
35,100001,KIA,Sportage,4500.0,Diésel,2008.0,128000,NaN,5,None,Manual,None,None,gris,Huelva,None,dataset kaggle,2021-01-14,202101,2021-01-15 00:00:00,9df64689ac5768546723b844d76a1b9b
39,100002,KIA,XCeed,32300.0,Híbrido enchufable,2021.0,2800,141.0,5,None,Manual,None,None,cassa,Murcia,None,dataset kaggle,2020-12-17,202012,2021-01-15 00:00:00,0d17809a4a1de7ef2c63f342f306713c
50,100003,NISSAN,NOTE,7900.0,Gasolina,2015.0,54200,80.0,5,None,Manual,None,None,blanco,Pontevedra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,a19d08cbe35ce508121bd5613d46455b
72,100004,HYUNDAI,i40,8400.0,Gasolina,2012.0,107000,135.0,4,None,Manual,None,None,gris,Álava,None,dataset kaggle,2021-01-08,202101,2021-01-15 00:00:00,02916bf1509d2988bc4f5cb9e2551799


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5463 entries, 9 to 49995
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        5463 non-null   int64  
 1   marca                5463 non-null   object 
 2   modelo               5463 non-null   object 
 3   precio               5463 non-null   float64
 4   combustible          5446 non-null   object 
 5   anyo_vehiculo        5462 non-null   float64
 6   kilometraje          5463 non-null   int64  
 7   potencia             4878 non-null   float64
 8   num_puertas          5463 non-null   int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          5452 non-null   object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                5425 non-null   object 
 14  provincia            5462 non-null   object 
 15  etiqueta_eco         0 non-null      objec

Antes de cargar los datos, debemos eliminar los 5 registros que tienen el campo ```"modelo"``` = *null*, ya que es una constraint de la tabla creada.

In [ ]:
df = df.dropna(subset=['modelo'])


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5463 entries, 9 to 49995
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        5463 non-null   int64  
 1   marca                5463 non-null   object 
 2   modelo               5463 non-null   object 
 3   precio               5463 non-null   float64
 4   combustible          5446 non-null   object 
 5   anyo_vehiculo        5462 non-null   float64
 6   kilometraje          5463 non-null   int64  
 7   potencia             4878 non-null   float64
 8   num_puertas          5463 non-null   int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          5452 non-null   object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                5425 non-null   object 
 14  provincia            5462 non-null   object 
 15  etiqueta_eco         0 non-null      objec

Vamos a guardar también este último DataFrame con la versión final de los datos sacados de Kaggle y el formato de nuestra tabla TX_VEHICULOS_SEG_MANO, por si tenemos que utilizarlo posteriormente.

In [35]:
df.to_csv("../include/data/processed/df_kaggle_db_nuevas_marcas.csv", index=False)

## Extra: cargar datos finales del DF de Kaggle en BBDD:

In [36]:
import pandas as pd

df = pd.read_csv("../include/data/processed/df_kaggle_db_nuevas_marcas.csv")

df.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,100000,NISSAN,NOTE,6300.0,Gasolina,2016.0,77000,80.0,5,NaN,Manual,NaN,NaN,blanco,Barcelona,NaN,dataset kaggle,2020-12-14,202012,2021-01-15 00:00:00,13b881bdd31a0387edd64edd726e8c93
1,100001,KIA,Sportage,4500.0,Diésel,2008.0,128000,NaN,5,NaN,Manual,NaN,NaN,gris,Huelva,NaN,dataset kaggle,2021-01-14,202101,2021-01-15 00:00:00,9df64689ac5768546723b844d76a1b9b
2,100002,KIA,XCeed,32300.0,Híbrido enchufable,2021.0,2800,141.0,5,NaN,Manual,NaN,NaN,cassa,Murcia,NaN,dataset kaggle,2020-12-17,202012,2021-01-15 00:00:00,0d17809a4a1de7ef2c63f342f306713c
3,100003,NISSAN,NOTE,7900.0,Gasolina,2015.0,54200,80.0,5,NaN,Manual,NaN,NaN,blanco,Pontevedra,NaN,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,a19d08cbe35ce508121bd5613d46455b
4,100004,HYUNDAI,i40,8400.0,Gasolina,2012.0,107000,135.0,4,NaN,Manual,NaN,NaN,gris,Álava,NaN,dataset kaggle,2021-01-08,202101,2021-01-15 00:00:00,02916bf1509d2988bc4f5cb9e2551799


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        5463 non-null   int64  
 1   marca                5463 non-null   object 
 2   modelo               5463 non-null   object 
 3   precio               5463 non-null   float64
 4   combustible          5446 non-null   object 
 5   anyo_vehiculo        5462 non-null   float64
 6   kilometraje          5463 non-null   int64  
 7   potencia             4878 non-null   float64
 8   num_puertas          5463 non-null   int64  
 9   num_plazas           0 non-null      float64
 10  tipo_cambio          5452 non-null   object 
 11  tipo_vehiculo        0 non-null      float64
 12  cilindrada_motor     0 non-null      float64
 13  color                5425 non-null   object 
 14  provincia            5462 non-null   object 
 15  etiqueta_eco         0 non-null      f

Antes de cargarlos, vamos a quitarle también los acentos a los valores de la columna modelo y a ponerlos en minúscula:

In [38]:
from unidecode import unidecode

df["modelo"] = df["modelo"].str.lower().apply(unidecode)

In [40]:
df[df["marca"] == "KIA"]["modelo"].value_counts()


modelo
sportage                   229
ceed                       141
carens                     115
stonic                      88
picanto                     71
rio                         63
xceed                       61
niro                        60
sorento                     53
carnival                    44
pro ceed                    34
niro phev                   25
optima                      23
ceed tourer                 20
soul                        15
ceed sportswagon            12
venga                       11
e-niro                      10
proceed                      8
optima sw                    7
cerato                       5
shuma                        4
stinger                      3
ceed sporty wagon            3
e-soul                       3
soul ev                      2
k2500                        1
optima phev                  1
leon                         1
sephia ii                    1
magentis                     1
pro_ceed gt                  1
n

In [41]:
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

try:
    df.to_sql('TX_VEHICULOS_SEG_MANO', con, if_exists='append', index=False)
    print("Datos insertados con éxito")
except Exception as e:
    print("Error al insertar datos:", e)

con.close()

Datos insertados con éxito


In [43]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT *
FROM TX_VEHICULOS_SEG_MANO
where
    --origen_anuncio = 'WebScraping'
    origen_anuncio = 'dataset kaggle'
    --AND marca not in ('BMW', 'VOLKSWAGEN', 'MERCEDES-BENZ', 'AUDI', 'PEUGEOT', 'FORD', 'RENAULT', 'OPEL', 'CITROEN', 'SEAT')

    --pk_anuncio_id in (1, 2, 3, 4)
order by
    fecha_carga desc
;
"""

df_db_new = pd.read_sql_query(query, con)

df_db_new.head()

,pk_anuncio_id,marca,modelo,precio,combustible,anyo_vehiculo,kilometraje,potencia,num_puertas,num_plazas,tipo_cambio,tipo_vehiculo,cilindrada_motor,color,provincia,etiqueta_eco,origen_anuncio,fecha_publicacion,anyomes_publicacion,fecha_carga,url
0,1,SEAT,toledo,950.0,Diésel,2000.0,227000,NaN,4,None,Manual,None,None,verde,Navarra,None,dataset kaggle,2020-12-18,202012,2021-01-15 00:00:00,e158ae0ca53119ca199c28c36b5c2fcd
1,2,CITROEN,c1,6200.0,Gasolina,2017.0,50071,82.0,5,None,Manual,None,None,blanco,Tenerife,None,dataset kaggle,2021-01-02,202101,2021-01-15 00:00:00,ff267ebb7e700246f47f84f3db660b4b
2,3,FORD,transit connect,7851.0,Diésel,2016.0,103000,100.0,4,None,Manual,None,None,blanco,Barcelona,None,dataset kaggle,2020-12-16,202012,2021-01-15 00:00:00,de4b02db28ea7786c622b969be10c7c7
3,4,VOLKSWAGEN,caravelle,19426.0,Diésel,2014.0,120000,140.0,4,None,Manual,None,None,blanco,Navarra,None,dataset kaggle,2020-11-25,202011,2021-01-15 00:00:00,0449972a4d07594acf92e9a7dd28b39c
4,5,FORD,transit,22850.0,Diésel,2017.0,107000,130.0,2,None,Manual,None,None,blanco,Sevilla,None,dataset kaggle,2021-01-12,202101,2021-01-15 00:00:00,12c4fa49bd4fdf23f19ecf396d3f02ef


In [44]:
df_db_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40504 entries, 0 to 40503
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pk_anuncio_id        40504 non-null  int64  
 1   marca                40504 non-null  object 
 2   modelo               40504 non-null  object 
 3   precio               40504 non-null  float64
 4   combustible          40469 non-null  object 
 5   anyo_vehiculo        40502 non-null  float64
 6   kilometraje          40504 non-null  int64  
 7   potencia             33143 non-null  float64
 8   num_puertas          40504 non-null  int64  
 9   num_plazas           0 non-null      object 
 10  tipo_cambio          40413 non-null  object 
 11  tipo_vehiculo        0 non-null      object 
 12  cilindrada_motor     0 non-null      object 
 13  color                40019 non-null  object 
 14  provincia            40499 non-null  object 
 15  etiqueta_eco         0 non-null     

Volvemos a guardar en .csv con el nuevo cambio:

In [45]:
df_db_new.to_csv("../include/data/processed/df_kaggle_final_db_nuevas_marcas.csv", index=False)

prueba:

In [46]:
import pandas as pd
import sqlite3

con = sqlite3.connect("../include/db_vehiculos.db")

query = """
SELECT marca, modelo
FROM TX_VEHICULOS_SEG_MANO
--where
    --origen_anuncio = 'dataset kaggle'
    --origen_anuncio = 'WebScraping'
group by
    marca, modelo
;
"""

df_db_modelo = pd.read_sql_query(query, con)

df_db_modelo.head()

,marca,modelo
0,AUDI,100
1,AUDI,80
2,AUDI,90
3,AUDI,a1
4,AUDI,a2


In [47]:
df_db_modelo["modelo"].value_counts()

modelo
coupe                                  2
transporter                            2
leon                                   2
100                                    1
108                                    1
zafira tourer                          1
zafira-e life                          1
1007                                   1
106                                    1
107                                    1
2008                                   1
zafira                                 1
205                                    1
206                                    1
206 +                                  1
206 sw                                 1
207                                    1
207 +                                  1
zafira life                            1
vectra                                 1
vivaro                                 1
208 xad                                1
tigra                                  1
signum                                 1
omega    